<a href="https://colab.research.google.com/github/CyberRik/house-prices-prediction/blob/main/house_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle


Importing the files


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


Reading the files, dropping target data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

ROUGH

In [ ]:
X_train.head()
X_train.shape
len(numerical_cols)
len(categorical_cols
    )

40

Importing necessary libraries


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import FunctionTransformer
from xgboost import XGBRegressor


Preprocessing

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median') # Your code here

# Preprocessing for categorical data
categorical_transformer =  Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])# Your code here

numerical_cols_filtered = X_train[numerical_cols].loc[:, X_train[numerical_cols].isnull().mean() <= 0.2].columns

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols_filtered),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [ ]:
#Using a XGBoost model
model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05)

#Defining the pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Fit the model with eval_set for early stopping (outside the pipeline)
my_pipeline.fit(X_train, y_train)

# Get predictions
predictions = my_pipeline.predict(X_valid)

# Calculate MAE
mae = mean_absolute_error(predictions, y_valid)

#print error
print(mae)

17512.341796875


Now for the test file


In [ ]:
pred = my_pipeline.predict(X_test)

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': pred})
output.to_csv('submission.csv', index=False)

Saving the model

In [ ]:
import joblib

# Suppose your pipeline is called 'pipeline'
joblib.dump(my_pipeline, 'house_price_pipeline.pkl')


['house_price_pipeline.pkl']

In [ ]:
from google.colab import files
files.download('house_price_pipeline.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
preprocessor = my_pipeline.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()

model = my_pipeline.named_steps['model']

# Get importance scores
importances = model.feature_importances_

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

# Sort by importance
importance_df = importance_df.sort_values(by='importance', ascending=False)

importance_df['raw_feature'] = importance_df['feature'].apply(
    lambda x: x.split('__')[-1].split('_')[0] if '__' in x and '_' in x else x.split('__')[-1]
)

raw_importance = importance_df.groupby('raw_feature')['importance'].sum().sort_values(ascending=False)

top_features = raw_importance.head(10)
print(top_features)



raw_feature
OverallQual     0.341962
GarageCars      0.095014
KitchenQual     0.075645
BsmtQual        0.043992
RoofStyle       0.042714
CentralAir      0.039777
MSZoning        0.034143
GrLivArea       0.030798
LotShape        0.028396
GarageFinish    0.023765
Name: importance, dtype: float32
